In [1]:
import pandas as pd
import numpy as np

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool

In [2]:
%autosave 0
%matplotlib inline
output_notebook()

Autosave disabled


Loading BokehJS ...

In [3]:
orig_df = pd.read_csv(r'C:\Users\germap01\Python\DATAVIZ\clusterly\iris.csv')
num_df = orig_df.select_dtypes(exclude='object')
Z = linkage(num_df, 'ward')

In [4]:
d = dendrogram(Z, no_plot=True)

In [5]:
for key, value in d.items():
    print(key, d[key][0])

icoord [25.0, 25.0, 35.0, 35.0]
dcoord [0.0, 0.14142135623730986, 0.14142135623730986, 0.0]
ivl 35
leaves 35
color_list g


In [8]:
#x axis (numerical)
xs = [val for key, val in d.items() if key == 'icoord'][0]

In [9]:
#distance
ys = [val for key, val in d.items() if key == 'dcoord'][0]

In [11]:
#original observation index (referring back to num_df) starting from left to right in the dendrogram
cl = [val for key, val in d.items() if key == 'ivl']
#convert strings to int
cl = [int(i) for i in cl[0]]

In [12]:
#circle coordinates for tooltips; adjust cx for getting the mid-point between two clusters
for key, value in d.items():
    if key == 'icoord':
        cx = [val[1] + ((val[2]-val[1]) / 2) for val in value]

for key, value in d.items():
    if key == 'dcoord':
        cy = [val[2] for val in value]

In [14]:
#circle coordinates for selection and tooltips; join on index

c_df = pd.DataFrame(data=dict(x = np.arange(5, len(cl) * 10 + 5, 10), y=0),
                    index = cl)

d_df = pd.merge(orig_df, c_df, how='left', left_index=True, right_index=True)

In [15]:
#draw the dendrogram with multiple renderers
p = figure(plot_width=800, plot_height=400, title='Zoomable dendrogram with tooltips')
p.title.align = 'center'

p.multi_line(xs=xs, ys=ys, line_width=1, line_color='black')

#invisible circles FTW!
c1 = p.circle(x=cx, y=cy, fill_color='black', size=10, fill_alpha=0, line_color=None)
c1_hover = HoverTool(renderers=[c1], tooltips=[('Distance', '@y'),])
c1_hover.toggleable = False
p.add_tools(c1_hover)

#add anohter circle renderer for the base of each cluster
source = ColumnDataSource(d_df)
c2 = p.circle(x='x', y='y', fill_color='black', size=10, fill_alpha=0, line_color=None, source=source)
c2_hover = HoverTool(renderers=[c2], tooltips=[('Species', '@species'),])
c2_hover.toggleable = False
p.add_tools(c2_hover)

p.grid.visible = False
p.xaxis.visible = False
p.yaxis.minor_tick_line_alpha = 0
p.yaxis.axis_label = 'Distance'

show(p)